In [1]:
from anonypy.anonypy import anonypy
import pandas as pd
import numpy as np

data_file_path = 'full_output.csv' 
df = pd.read_csv(data_file_path)

categorical = set(("test","Outcome"))  

D:\software\Anaconda\anaconda\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
D:\software\Anaconda\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df

,test,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,6,148,72.0,35,169.5,33.6,0.627,50,1
1,1,1,85,66.0,29,102.5,26.6,0.351,31,0
2,1,8,183,64.0,32,169.5,23.3,0.672,32,1
3,1,1,89,66.0,23,94.0,28.1,0.167,21,0
4,1,5,137,40.0,35,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...,...
763,1,10,101,76.0,48,180.0,32.9,0.171,63,0
764,1,2,122,70.0,27,102.5,36.8,0.340,27,0
765,1,5,121,72.0,23,112.0,26.2,0.245,30,0
766,1,1,126,60.0,32,169.5,30.1,0.349,47,1


In [3]:
for name in categorical:
    df[name] = df[name].astype("category")

feature_columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
sensitive_column = "Outcome"

p = anonypy.Preserver(df, feature_columns, sensitive_column)
rows = p.anonymize_k_anonymity(k=16)

dfn = pd.DataFrame(rows)

E:\jupyter\eps\anonypy\anonypy\anonypy.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.loc[partition].groupby(sensitive_column).agg({sensitive_column: "count"})


In [4]:
dfn

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,count
0,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0,20
1,1-2,75-139,30.0-66.0,16-52,16.0-99.0,28.4-55.0,0.166-1.251,21-33,0,18
2,1-2,75-139,30.0-66.0,16-52,16.0-99.0,28.4-55.0,0.166-1.251,21-33,1,2
3,1-2,68-119,68.0-90.0,29-52,18.0-220.0,25.0-46.3,0.127-1.096,21-52,0,23
4,1-2,68-119,68.0-90.0,29-52,18.0-220.0,25.0-46.3,0.127-1.096,21-52,1,8
...,...,...,...,...,...,...,...,...,...,...
73,4-14,90-129,58.0-90.0,25-49,125.0-230.0,32.5-46.8,0.378-1.39,34-56,1,15
74,7-13,133-197,50.0-90.0,27-46,29.0-392.0,25.9-52.3,0.383-1.191,34-43,0,5
75,7-13,133-197,50.0-90.0,27-46,29.0-392.0,25.9-52.3,0.383-1.191,34-43,1,17
76,7-17,130-196,60.0-110.0,15-48,60.0-495.0,25.9-45.4,0.443-1.441,45-81,0,5


In [5]:
new_dfn = pd.DataFrame()

for _, row in dfn.iterrows():
    # 根据count列的值复制该行count次
    new_rows = pd.DataFrame([row.values] * row['count'], columns=dfn.columns)
    
    # 将新行添加到新的DataFrame中
    new_dfn = pd.concat([new_dfn, new_rows], ignore_index=True)
    
    # 将新行的count值设置为1
    new_dfn.loc[new_dfn.shape[0]-row['count']:, 'count'] = 1

In [6]:
new_dfn = new_dfn.drop('count', axis=1)

In [7]:
new_dfn.to_csv('new_dfn.csv', index=False)

In [8]:
new_dfn

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0
1,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0
2,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0
3,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0
4,1-2,56-119,24.0-66.0,12-35,15.0-100.0,18.2-28.1,0.128-1.268,21-26,0
...,...,...,...,...,...,...,...,...,...
763,7-17,130-196,60.0-110.0,15-48,60.0-495.0,25.9-45.4,0.443-1.441,45-81,1
764,7-17,130-196,60.0-110.0,15-48,60.0-495.0,25.9-45.4,0.443-1.441,45-81,1
765,7-17,130-196,60.0-110.0,15-48,60.0-495.0,25.9-45.4,0.443-1.441,45-81,1
766,7-17,130-196,60.0-110.0,15-48,60.0-495.0,25.9-45.4,0.443-1.441,45-81,1


In [9]:
new_dfn.to_csv('new_dfn.csv', index=False)

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [158]:
# 类别型特征列表
cat_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# 对类别型特征进行one-hot编码
data_encoded = pd.get_dummies(new_dfn, columns=cat_features)

# 分离特征和标签
X = data_encoded.drop('Outcome', axis=1)
y = data_encoded['Outcome']

# 对数值型特征进行归一化处理
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 合并归一化后的特征和标签
preprocessed_data = pd.concat([X_scaled, y], axis=1)

In [159]:
preprocessed_data

,Pregnancies_1-2,Pregnancies_3-10,Pregnancies_3-11,Pregnancies_3-12,Pregnancies_3-13,Pregnancies_3-5,Pregnancies_3-6,Pregnancies_3-7,Pregnancies_3-8,Pregnancies_3-9,...,Age_34-67,Age_34-68,Age_34-69,Age_37-65,Age_37-70,Age_39-66,Age_42-65,Age_45-81,Age_49-69,Outcome
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
